In [1]:
#!pip install dash
#!pip install dash-extensions

In [2]:
from dash import Dash, dcc, html, Input, Output, callback, State
import json
import plotly.graph_objects as go
import pandas as pd
import pandas_ta as ta
import forecast
from plotly.subplots import make_subplots

candles_df = forecast.get_df('BTCUSD', '2024-01-02 00:00', '1T')
df = candles_df[0:90].copy()

# Вычисляем индикаторы с помощью pandas_ta
df['SMA20'] = ta.sma(df['close'], length=20)
df['EMA20'] = ta.ema(df['close'], length=20)

app = Dash(__name__)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03,
               row_heights=[0.8, 0.2],
               specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

# Добавляем свечной график
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='Candlesticks'), row=1, col=1)

# Добавляем объемы во второй ряд
fig.add_trace(go.Bar(x=df.index, y=df['volume']), row=2, col=1)

# Настраиваем внешний вид графика
fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    margin=dict(
        l=60, # нужен запас, чтобы подписи влазили. иначе будет дергаться график при зумах.
        r=0,
        b=0,
        t=20, # нужен запас, чтобы подписи влазили. иначе будет дергаться график при зумах.
        pad=0
    ),
    xaxis=dict(
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='solid',
        spikethickness=1,
    ),
    xaxis2=dict(
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='solid',
        spikethickness=1,
    ),
    hovermode='x',
    dragmode='pan'  # Устанавливаем режим панорамирования по умолчанию
)

fig.update(layout_xaxis_rangeslider_visible=False)

app.layout = html.Div([
    dcc.Graph(
        id='basic-interactions',
        figure=fig,
        config={
            'scrollZoom': True,
            'displayModeBar': True,
            'modeBarButtonsToAdd': ['pan2d'],
            'displaylogo': False,
        }
    ),
    dcc.Store(id='click-store', data=[]),
    dcc.Store(id='relayout-data-store'),
    html.Pre(id='output-container', style={
        'background-color': 'white', 
        'position': 'absolute', 
        'top': '200px', 
        'left': f'{fig.layout.width - 180}px'
    })
])

@callback(
    Output('basic-interactions', 'figure'),
    Input('basic-interactions', 'clickData'),
    [State('click-store', 'data'),
     State('basic-interactions', 'relayoutData')])
def update_graph_on_click(clickData, click_store, relayout_data):
    if clickData is None:
        return fig

    click_store = click_store or []
    x_value = clickData['points'][0]['x']
    # Получаем строку из DataFrame по выбранному значению x_value
    row = df.loc[df.index == x_value]

    # # Печатаем все значения строки
    # print(f"Data for {x_value}:")
    # print(row.to_string(index=False))
    
    # Если есть сохраненное состояние осей, применяем его
    if relayout_data and not relayout_data.get('xaxis.range[0]', None) == None:
        # print(json.dumps(relayout_data, indent=2))
        # print(json.dumps(relayout_data.get('xaxis.range[0]', {}), indent=2))
        fig.update_layout(xaxis=dict(range=[relayout_data.get('xaxis.range[0]'), relayout_data.get('xaxis.range[1]')]),
                          yaxis=dict(range=[relayout_data.get('yaxis.range[0]'), relayout_data.get('yaxis.range[1]')]),
                          xaxis2=dict(range=[relayout_data.get('xaxis2.range[0]'), relayout_data.get('xaxis2.range[1]')]),
                          yaxis2=dict(range=[relayout_data.get('yaxis2.range[0]'), relayout_data.get('yaxis2.range[1]')]))

    # Добавляем вертикальную линию
    click_store.append({
        'type': 'line',
        'xref': 'x',
        'yref': 'paper',
        'x0': x_value,
        'y0': 0,
        'x1': x_value,
        'y1': 1,
        'line': {
            'color': 'black',
            'width': 1,
            'dash': 'dash'
        }
    })

    fig.update_layout(shapes=click_store)


    return fig

@callback(
    Output('output-container', 'children'),
    Input('basic-interactions', 'clickData'))
def update_output_div(clickData):
    if clickData is None:
        return "No click data available"
    
    x_value = clickData['points'][0]['x']
    # Получаем строку из DataFrame по выбранному значению x_value
    row = df.loc[df.index == x_value]

    # Создаем строку для вывода информации
    output_string = f"date: {x_value}:"
    for col in row.columns:
        value = row[col].values[0]
        output_string += f'\n{col}:{value:.2f}' if isinstance(value, (int, float)) else f'\n{col}:{value}'

    return output_string

@callback(
    Output('relayout-data-store', 'data'),
    Input('basic-interactions', 'relayoutData'))
def store_relayout_data(relayout_data):
    return relayout_data

if __name__ == "__main__":
    app.run_server(debug=True)
